In [1]:
import pandas as pd
import os
import string
import spacy
import numpy as np
import array
import itertools
import json
import argparse
import time
from parse_and_split import *

In [2]:
eng = spacy.en.English()

In [3]:
data = "reviews_Musical_Instruments_5.json.gz"
start = time.clock()
words_freq = get_frequent_words(get_rev(data, 'reviewText'), 10, eng)
# somehow the empty string is included, mabey empty reviews?
words_freq = [(w, count) for w, count in words_freq if w != 0]
words, counts = zip(*words_freq)
end = time.clock()
print "build vocab done", end - start

start = time.clock()

# get word frequency per review
bags = get_word_bags(words, get_rev(data, 'reviewText'), eng)

end = time.clock()
print "word bags done", end - start
# parse dataframe
df = getDF(data)

    # delete useless stuff
df.drop(['reviewText', 'reviewerName', 'helpful', 'unixReviewTime', 'overall', 'reviewTime', 'summary'], axis=1, inplace=True)

    # put bags into pd df
wbdf = pd.DataFrame({'wordBags':bags})

    # unique lists
users = df['reviewerID'].sort_values().unique()
prods = df['asin'].sort_values().unique()

userkeys = {u: i for i, u in enumerate(users)}
prodkeys = {p: i for i, p in enumerate(prods)}

    # df used to split in user and prod dimensions
df_total = pd.concat([df,wbdf], axis=1, join='inner', copy='false')

build vocab done 7.919095
word bags done 6.459518


In [45]:
tmp_samples = df_total.sample(frac=0.5, random_state=np.random.RandomState())

In [4]:
def split_train_test(df, frac=0, n=0):
    train = df
    if frac != 0:
        test = train.sample(frac=frac, random_state=np.random.RandomState())
    else:
        test = train.sample(n=n, random_state=np.random.RandomState())
    train = train.drop(test.index)
    
    miss_user = [user for user in test['reviewerID'] if user not in train['reviewerID'].values]
    miss_prod = [prod for prod in test['asin'] if prod not in train['asin'].values]
    
    put_back = test[test['reviewerID'].isin(miss_user) & test['asin'].isin(miss_prod)]
    print len(put_back)
    
    miss_user = [user for user in miss_user if user not in put_back['reviewerID'].values]
    miss_prod = [prod for prod in miss_prod if prod not in put_back['asin'].values]
    
    print len(miss_user), len(miss_prod)
    print len(train), len(test)
    for mu in miss_user:
        tmp = test[test['reviewerID'] == mu].sample(n=1, random_state=np.random.RandomState())
        put_back.append(tmp)
    for mp in miss_prod:
        tmp = test[test['asin'] == mp].sample(n=1, random_state=np.random.RandomState())
        put_back.append(tmp)
    
    return pd.concat([train, put_back]), test.drop(put_back.index)
        
        

In [9]:
train, test = split_train_test(df_total, frac=0.2)

0
0 0
8209 2052


10261

In [106]:
test

,reviewerID,asin,wordBags
3925,A2XD36ONYLNU5J,B000978D58,"{39: 1, 2172: 1, 266: 1, 14: 2, 16: 1, 1585: 1..."
4211,A3IDC5FP52KJWJ,B000B6DHB2,"{355: 1, 230: 1, 17: 1, 200: 1, 974: 1, 15: 2,..."
6660,A36P367G4ZH6Z0,B001L8MDMS,"{12: 1, 21: 1, 2198: 1, 37: 1, 1244: 1, 2220: ..."
1310,A34BZPTKSY68PB,B0002E1G5C,"{58: 1, 990: 1}"
2877,A149F91FB9WTW6,B0002H0A3S,"{1409: 3, 442: 1, 966: 1, 161: 1, 974: 1, 60: ..."
8330,A27UAE2PME5TRV,B003VWKPHC,"{513: 2, 2056: 2, 2057: 1, 12: 1, 13: 3, 14: 1..."
7481,A14CQY582WMP7O,B002VA464S,"{513: 1, 2307: 1, 65: 1, 12: 3, 13: 1, 14: 10,..."
6716,A341MNU9GWBWKS,B001MMYVZU,"{1409: 1, 519: 1, 1290: 1, 12: 1, 13: 1, 14: 4..."
502,A306YUQBEB2R9T,B0002CZVWS,"{1890: 2, 2710: 1, 39: 1, 13: 1, 15: 1, 2804: ..."
5231,AE9C0UNXBV8CB,B000OR88JE,"{1793: 1, 1122: 1, 484: 1, 1409: 1, 1161: 1, 1..."
